In [67]:
import numpy as np
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import matplotlib.pyplot as plt
import matplotlib.ticker
import math
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': print(torch.cuda.get_device_name()) 

cpu


# Data Prep

Training and Testing data is prepared from the solution file

In [68]:
x_1 = np.linspace(0,1,128)  # 256 points between -1 and 1 [256x1]
x_2 = np.linspace(1,0,128)  # 256 points between 1 and -1 [256x1]
x_3 = np.linspace(0,1,128)  # 256 points between -1 and 1 [256x1]

X, Y, Z = np.meshgrid(x_1,x_2,x_3)

# Test Data

We prepare the test data to compare against the solution produced by the SSDRM.

In [ ]:
X_v_test = np.hstack((X.flatten(order='F')[:,None], Y.flatten(order='F')[:,None],Z.flatten(order='F')[:,None]))

lb = np.array([0, 0, 0]) #lower bound
ub = np.array([1, 1, 1])  #upper bound

usol = 1/(4 * np.pi)*Z*np.log((np.sqrt(X**2 + Y**2+(Z-0.8)**2)-(Z-0.8))/(np.sqrt(X**2 + Y**2+(Z-0.2)**2)-(Z-0.2)))-np.sqrt(X**2 + Y**2+(Z-0.2)**2)+np.sqrt(X**2 + Y**2+(Z-0.8)**2)
vsol = -np.sqrt(X**2 + Y**2+(Z-0.2)**2)+np.sqrt(X**2 + Y**2+(Z-0.8)**2)

v_true = vsol.flatten('F')[:,None] 

# Training Data

In [70]:
def trainingdata(N_v,N_f):
    
    leftedge_x = np.hstack((X[0,:].flatten('a')[:,None], Y[0,:].flatten('a')[:,None], Z[0,:].flatten('a')[:,None]))
    leftedge_v = vsol[0,:].flatten('a')[:,None]
    
    rightedge_x = np.hstack((X[-1,:].flatten('a')[:,None], Y[-1,:].flatten('a')[:,None], Z[-1,:].flatten('a')[:,None]))
    rightedge_v = vsol[-1,:].flatten('a')[:,None]
    
    backedge_x = np.hstack((X[:,0].flatten('a')[:,None], Y[:,0].flatten('a')[:,None], Z[:,0].flatten('a')[:,None]))
    backedge_v = vsol[:,0].flatten('a')[:,None]
    
    frontedge_x = np.hstack((X[:,-1].flatten('a')[:,None], Y[:,-1].flatten('a')[:,None], Z[:,-1].flatten('a')[:,None]))
    frontedge_v = vsol[:,-1].flatten('a')[:,None]
    
    bottomedge_x = np.hstack((X[:,:,0].flatten('a')[:,None], Y[:,:,0].flatten('a')[:,None], Z[:,:,0].flatten('a')[:,None]))
    bottomedge_v = vsol[:,:,0].flatten('a')[:,None]
    
    topedge_x = np.hstack((X[:,:,-1].flatten('a')[:,None], Y[:,:,-1].flatten('a')[:,None], Z[:,:,-1].flatten('a')[:,None]))
    topedge_v = vsol[:,:,-1].flatten('a')[:,None]
    
    all_X_v_train = np.vstack([leftedge_x, rightedge_x, backedge_x, frontedge_x, bottomedge_x, topedge_x])
    all_v_train = np.vstack([leftedge_v, rightedge_v, backedge_v, frontedge_v, bottomedge_v, topedge_v])
     
    #choose random N_v points for training
    idx = np.random.choice(all_X_v_train.shape[0], N_v, replace=False) 
    
    X_v_train = all_X_v_train[idx[0:N_v], :] #choose indices from  set 'idx' (x,t)
    v_train = all_v_train[idx[0:N_v],:]      #choose corresponding v
    
    '''Collocation Points'''

    # N_f sets of tuples(x,t)
    X_f = lb + (ub-lb)*lhs(3,N_f)
    X_f_train = np.vstack((X_f, X_v_train)) # append training points to collocation points 
    
    return X_f_train, X_v_train, v_train 

# SSDRM

Creating sequential layers using the class
tf.Module

In [71]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
    
        'Initialise neural network as a nn.MSELosslist using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
    
        'Xavier Normal Initialization'
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            
            # weights from a normal distribution with 
            # Recommended gain value for tanh = 5/3?
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            

    def forward(self,x):
        
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            
            z = self.linears[i](a)
                        
            a = self.activation(z)
            
        a = self.linears[-1](a)
        
        return a
                        
    def loss_BC(self,x,y):
                
        loss_v = self.loss_function(self.forward(x), y)
                
        return loss_v
    
    def loss_PDE(self, x_to_train_f):
                
        x_1_f = x_to_train_f[:,[0]]
        x_2_f = x_to_train_f[:,[1]]
        x_3_f = x_to_train_f[:,[2]]
                        
        g = x_to_train_f.clone()
                        
        g.requires_grad = True
        
        v = self.forward(g)
                
        v_x = autograd.grad(v,g,torch.ones([x_to_train_f.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
                                                            
        v_x_1 = v_x[:,[0]]
        
        v_x_2 = v_x[:,[1]]
        
        v_x_3 = v_x[:,[2]]
                        
        k = 1
        F = 2/np.sqrt(x_1_f**2 + x_2_f**2+(x_3_f-0.2)**2)-2/np.sqrt(x_1_f**2 + x_2_f**2+(x_3_f-0.8)**2)
        
        f = 1/2 * k * (v_x_1**2 + v_x_2**2 + v_x_3**2) - F * v

        loss_f = torch.mean(f)
                
        return loss_f
    
    def loss(self,x,y,x_to_train_f,sigma):

        loss_v = self.loss_BC(x,y)
        loss_f = self.loss_PDE(x_to_train_f)

        loss = sigma/2 * loss_v + loss_f

        return loss
     
    'callable for optimizer'                                       
    def closure(self):
        
        optimizer.zero_grad()
        
        loss_val = self.loss(X_v_train, v_train, X_f_train, sigma)
        
        # error_vec, _ = SSDRM.test()
        
        global ite, iteration_vec, fun_vec, err_vec
        ite = ite + 1
        
        if (ite % 50 == 0):
            iteration_vec.append(ite)
            fun_vec.append(loss_val.item())
            error_vec, _ = SSDRM.test()
            err_vec.append(error_vec.detach().float().item())
        
        loss_val.backward()

        return loss_val        
    
    def test(self):
                
        v_pred = self.forward(X_v_test_tensor)
        
        error_vec = torch.linalg.norm((v-v_pred),2)/torch.linalg.norm(v,2)        # Relative L2 Norm of the error (Vector)
        
        v_pred = np.reshape(v_pred.cpu().detach().numpy(),(128,128,128),order='F') 
        
        return error_vec, v_pred

# Loss Function

The loss function consists of two parts:

    loss_BC: MSE error of boundary losses
    loss_PDE: variational functional for the PDE

In [72]:
N_v = 1200 
N_f = 20000 

X_f_train_np_array, X_v_train_np_array, v_train_np_array = trainingdata(N_v,N_f)

'Convert to tensor and send to GPU'
X_f_train = torch.from_numpy(X_f_train_np_array).float().to(device)
X_v_train = torch.from_numpy(X_v_train_np_array).float().to(device)
v_train = torch.from_numpy(v_train_np_array).float().to(device)
X_v_test_tensor = torch.from_numpy(X_v_test).float().to(device)
v = torch.from_numpy(v_true).float().to(device)

iteration_vec = []
err_vec = []
fun_vec = []

layers = np.array([3,10,10,1])

SSDRM = Sequentialmodel(layers)
       
SSDRM.to(device)

'Neural Network Summary'

print(SSDRM)
params = list(SSDRM.parameters())

sigma = 8000
start_time = time.time()

ite = 0
optimizer = torch.optim.LBFGS(SSDRM.parameters(), lr=0.05, 
                                  max_iter = 1000, 
                                  max_eval = 2500, 
                                  tolerance_grad = 1e-06, 
                                  tolerance_change = 1e-09, 
                                  history_size = 100, 
                                  line_search_fn = 'strong_wolfe')

optimizer.zero_grad()     # zeroes the gradient buffers of all parameters
optimizer.step(SSDRM.closure)
error_vec, _ = SSDRM.test()
    
elapsed = time.time() - start_time                
print('Training time: %.2f' % (elapsed))


''' Model Accuracy ''' 
error_vec, v_pred = SSDRM.test()

print('sigma: %f' %(sigma))
print('Test Error: %.5f'  % (error_vec))

u_pred = 1/(4 * np.pi)*Z*np.log((np.sqrt(X**2 + Y**2+(Z-0.8)**2)-(Z-0.8))/(np.sqrt(X**2 + Y**2+(Z-0.2)**2)-(Z-0.2))) + v_pred

print(iteration_vec)
print(fun_vec)
print(err_vec)

Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=1, bias=True)
  )
)
Training time: 41.26
sigma: 8000.000000
Test Error: 0.02229
[50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700, 1750, 1800, 1850, 1900, 1950]
[6.860057353973389, 2.506141185760498, 2.062067747116089, 1.8426717519760132, 0.7765082120895386, 0.4400440454483032, 0.2722795903682709, 0.20307712256908417, 0.16022911667823792, 0.13159331679344177, 0.10386569797992706, 0.0801507979631424, 0.06713522225618362, 0.05460882931947708, 0.0466846227645874, 0.03865143284201622, 0.03186650201678276, 0.023840077221393585, 0.016639992594718933, 0.0115143321454525, 0.007077079266309738, 0.0038492754101

C:\Users\HP\AppData\Local\Temp/ipykernel_2632/3272857609.py:54: RuntimeWarning: divide by zero encountered in true_divide
  u_pred = 1/(4 * np.pi)*Z*np.log((np.sqrt(X**2 + Y**2+(Z-0.8)**2)-(Z-0.8))/(np.sqrt(X**2 + Y**2+(Z-0.2)**2)-(Z-0.2))) + v_pred
C:\Users\HP\AppData\Local\Temp/ipykernel_2632/3272857609.py:54: RuntimeWarning: invalid value encountered in true_divide
  u_pred = 1/(4 * np.pi)*Z*np.log((np.sqrt(X**2 + Y**2+(Z-0.8)**2)-(Z-0.8))/(np.sqrt(X**2 + Y**2+(Z-0.2)**2)-(Z-0.2))) + v_pred
